# Phyling API

Créez un fichier .env avec ce contenu :

```env
PHYLING_MAIL=your_email
PHYLING_PASSWORD=your_password
```

In [ ]:
import json
import logging
import os
import sys
from dotenv import load_dotenv

load_dotenv()

path = os.getenv("PHYLING_PATH")
if path:
  sys.path.append(path)

from phyling.api import PhylingAPI


# Connection a l'API

In [ ]:
api = PhylingAPI(
    mail=os.getenv("PHYLING_MAIL"),
    password=os.getenv("PHYLING_PASSWORD"),
    url="https://api.app.phyling.fr/"
)
logging.info(api)

# Exemples

## Récupérer les données de l'utilisateur connecté

In [ ]:
res = api.GET(
    url="/login"
)
connectedUser = json.loads(res.data.decode("utf-8"))
print(connectedUser)


## Récupérer des utilisateurs

Une liste de filtres peuvent êtres appliqués :
```python
def get_users(
    self,
    client_id: Union[int, None] = None,
    group_ids: list = [],
    role: Union[str, None] = None,
    active: bool = True,
    search: str = "",
    pageId: int = 1,
    pageSize: int = 0,
    soft: bool = False,
) -> Union[dict, None]:
    """Get the users from the API

    Args:
        client_id (int): clients id. None to select all
        group_ids (list): list of group ids. Empty to select all
        role (str, optional): The users roles (Admin, Coach or Athlete). None to select all
        active (bool, optional): The users active status. Defaults to True.
        search (str): The search string
        pageSize (int): Size of one page. Default: -1 (select all)
        pageId (int): Page id. Default: 1
        soft (bool): If true, only return minimal information on users

    Returns:
        dict: {
            "items": [User, ...],
            "total": int,
        }
    """
    pass
```

In [ ]:
users = api.get_users(
    client_id=connectedUser["client_id"],
)
print(users)

In [ ]:
print(users["items"][0].to_dict())

## Récupérer des records

Une liste de filtres peuvent êtres appliqués :
```python
def get_records(
    self,
    type: str = "all",
    pageSize: int = 10,
    pageId: int = 1,
    onlyFavorite: bool = False,
    userIds: list = [],
    deviceIds: list = [],
    clientIds: list = [],
    groupIds: list = [],
    exerciseIds: list = [],
    sportIds: list = [],
    record_type: str = "",
    scenarioIds: list = [],
    minDate: str = "",
    maxDate: str = "",
) -> Union[dict, None]:
    """Get the records from the API

    Args:
        type (str): should be `all`, `new` or `associated`
        pageSize (int): Size of one page. Default: -1 (select all)
        pageId (int): Page id. Default: 1
        onlyFavorite (bool, optional): Select only favorites. Default to False.
        userIds (list): Users ids. Empty to select all
        deviceIds (list): devices ids. Empty to select all
        clientIds (list): clients ids. Empty to select all
        groupIds (list): groups ids. Empty to select all
        exerciseIds (list): exercises ids. Empty to select all
        sportIds (list): sport ids. Empty to select all
        record_type (str): record type. Can be seance, record, scenario, calib, miniphyling, fusion or video
        scenarioIds (list[int]): scenario ids. Empty to select all
        minDate (str): min record date. Empty to select all
        maxDate (str): max record date. Empty to select all

    Returns:
        dict: {
            "records": [Record, ...],
            "total": int,
        }
    """
    pass
```

In [ ]:
recs = api.get_records(
    userIds=[connectedUser["id"]],
)

In [ ]:
recs

In [ ]:
print(recs["records"][0])

---
Téléchargement de record
```python
def download_record(
    self,
    rec_id: int,
    file_type: str,
    download_path: str,
    overwrite: bool = True,
    timeout: int = 180,
    **kwargs,
) -> bool:
    """
    Download the record.

    Args:
        rec_id (int): The ID of the record to download.
        file_type (str): The type of file to download. Can be one of the following:
            - "raw": raw data (.txt)
            - "decoded": decoded data (.csv)
            - "pdf": pdf report (.pdf)
            - "stats": stats report (.csv)
            - "specific_stats": specific stats report (.csv)
            - "compressed": compressed data (.csv)
            - "video": video file (.mp4) -> only for video records
            - "zip": zip file (.zip)
        download_path (str): The path to save the downloaded file.
        overwrite (bool): If True, overwrite the file if it already exists. Default is False.
        timeout (int): The timeout for the request. Default is 180 seconds.
        **kwargs: Additional arguments for the request.
    """
    pass
```

In [ ]:
api.download_record(
    rec_id=recs["records"][0]["id"],
    file_type="raw",
    download_path="/tmp/example_data.txt",
    overwrite=True,
)

---
Décoder un record RAW

In [ ]:
import os
from phyling.decoder import decoder

In [ ]:
path = "/tmp/"
raw_file = "example_data.txt"
data_json = decoder.decode(os.path.join(path, raw_file), verbose=True, use_s3=False)

In [ ]:
# list modules
print(data_json["modules"].keys())

---
Process and plot

In [ ]:
# Library imports
%matplotlib widget
import matplotlib.pyplot as plt
from phyling import utils

In [ ]:
# Directly from data object

modName = "imu"
mod = data_json["modules"][modName]["data"]

plt.figure()
plt.plot(mod["T"], mod["acc_x"])
plt.xlabel("Temps (s)")
plt.ylabel("Acc. (m/s$^2$)")

In [ ]:
# Convert data object to Dataframes

df = utils.data2df(data_json, "all")
print(list(df))
df["imu"].head()

In [ ]:
plt.figure()
plt.plot(df["imu"]["T"], df["imu"]["acc_x"])
plt.xlabel("Temps (s)")
plt.ylabel("Acc. (m/s$^2$)")